<a href="https://colab.research.google.com/github/JXLK-dev/Toxic_classification_app/blob/master/MACHINE_LEARNING_PROJECT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -q kaggle

In [2]:
from google.colab import files
files.upload()
! mkdir ~/.kaggle
! cp kaggle.json ~/.kaggle/
! chmod 600 ~/.kaggle/kaggle.json

Saving kaggle.json to kaggle.json


In [3]:
! kaggle competitions download -c jigsaw-toxic-comment-classification-challenge
! unzip jigsaw-toxic-comment-classification-challenge.zip
!unzip sample_submission.csv.zip
!unzip test.csv.zip
!unzip test_labels.csv.zip
!unzip train.csv.zip

 70% 37.0M/52.6M [00:00<00:00, 170MB/s] 
100% 52.6M/52.6M [00:00<00:00, 171MB/s]
Archive:  jigsaw-toxic-comment-classification-challenge.zip
  inflating: sample_submission.csv.zip  
  inflating: test.csv.zip            
  inflating: test_labels.csv.zip     
  inflating: train.csv.zip           
Archive:  sample_submission.csv.zip
  inflating: sample_submission.csv   
Archive:  test.csv.zip
  inflating: test.csv                
Archive:  test_labels.csv.zip
  inflating: test_labels.csv         
Archive:  train.csv.zip
  inflating: train.csv               


In [4]:
import matplotlib.pyplot as plt
import nltk
import numpy as np
import re
import pandas as pd
import string
import seaborn as sns

from nltk.corpus import stopwords as sw
from nltk.stem.lancaster import LancasterStemmer as LS

from sklearn.feature_extraction.text import CountVectorizer as CV, TfidfVectorizer as TV

In [5]:
sample = pd.read_csv('/content/sample_submission.csv')
test = pd.read_csv('/content/test.csv')
test_label = pd.read_csv('/content/test_labels.csv')
train = pd.read_csv('/content/train.csv')


In [ ]:
train.head()

,id,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate
0,0000997932d777bf,Explanation\nWhy the edits made under my usern...,0,0,0,0,0,0
1,000103f0d9cfb60f,D'aww! He matches this background colour I'm s...,0,0,0,0,0,0
2,000113f07ec002fd,"Hey man, I'm really not trying to edit war. It...",0,0,0,0,0,0
3,0001b41b1c6bb37e,"""\nMore\nI can't make any real suggestions on ...",0,0,0,0,0,0
4,0001d958c54c6e35,"You, sir, are my hero. Any chance you remember...",0,0,0,0,0,0


In [6]:
!pip install nltk

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [ ]:
dataCount = train.iloc[:,2:].sum()
dataCount

toxic            15294
severe_toxic      1595
obscene           8449
threat             478
insult            7877
identity_hate     1405
dtype: int64

In [ ]:
# len(train)
# print(train)
# sample
train
# test
# test_label


,id,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate
0,0000997932d777bf,explanation why the edits made under my userna...,0,0,0,0,0,0
1,000103f0d9cfb60f,d aww he matches this background colour i m s...,0,0,0,0,0,0
2,000113f07ec002fd,hey man i m really not trying to edit war it...,0,0,0,0,0,0
3,0001b41b1c6bb37e,more i can t make any real suggestions on im...,0,0,0,0,0,0
4,0001d958c54c6e35,you sir are my hero any chance you remember...,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...
159566,ffe987279560d7ff,and for the second time of asking when ...,0,0,0,0,0,0
159567,ffea4adeee384e90,you should be ashamed of yourself that is a ...,0,0,0,0,0,0
159568,ffee36eab5c267c9,spitzer umm theres no actual article for pr...,0,0,0,0,0,0
159569,fff125370e4aaaf3,and it looks like it was actually you who put ...,0,0,0,0,0,0


In [7]:
# transform all text into alphabets only by turning unnecessary characters into space
alphanum = lambda x: re.sub('\w*\d\w*', ' ', x)
lower_punc = lambda x: re.sub('[%s]' % re.escape(string.punctuation), ' ', x.lower())
enter = lambda x: re.sub("\n", " ", x)
non_ascii = lambda x: re.sub(r'[^\x00-\x7f]',r' ', x)
train['comment_text'] = train['comment_text'].map(alphanum).map(lower_punc).map(enter).map(non_ascii)

In [8]:
#Divide datasets into 6 boolean categories
toxic = train.loc[:,['id','comment_text','toxic']]
severe = train.loc[:,['id','comment_text','severe_toxic']]
obscene = train.loc[:,['id','comment_text','obscene']]
threat = train.loc[:,['id','comment_text','threat']]
insult = train.loc[:,['id','comment_text','insult']]
identity_hate = train.loc[:,['id','comment_text','identity_hate']]


In [9]:
toxic_1 = toxic[toxic['toxic'] == 1].iloc[0:5000,:]
toxic_0 = toxic[toxic['toxic'] == 0].iloc[0:5000,:]
toxic_done = pd.concat([toxic_1, toxic_0], axis=0)
severe_1 = severe[severe['severe_toxic'] == 1].iloc[0:1595,:]
severe_0 = severe[severe['severe_toxic'] == 0].iloc[0:1595,:]
severe_done = pd.concat([severe_1, severe_0], axis=0)
obscene_1 = obscene[obscene['obscene'] == 1].iloc[0:5000,:]
obscene_0 = obscene[obscene['obscene'] == 0].iloc[0:5000,:]
obscene_done = pd.concat([obscene_1, obscene_0], axis=0)
threat_1 = threat[threat['threat'] == 1].iloc[0:478,:]
threat_0 = threat[threat['threat'] == 0].iloc[0:1912,:]  
threat_done = pd.concat([threat_1, threat_0], axis=0)
insult_1 = insult[insult['insult'] == 1].iloc[0:5000,:]
insult_0 = insult[insult['insult'] == 0].iloc[0:5000,:]
insult_done = pd.concat([insult_1, insult_0], axis=0)
identity_hate_1 = identity_hate[identity_hate['identity_hate'] == 1].iloc[0:1405,:]
identity_hate_0 = identity_hate[identity_hate['identity_hate'] == 0].iloc[0:5620,:]
identity_hate_done = pd.concat([identity_hate_1, identity_hate_0], axis=0)

In [10]:
toxic_done.head()

,id,comment_text,toxic
6,0002bcb3da6cb337,cocksucker before you piss around on my work,1
12,0005c987bdfc9d4b,hey what is it talk what is it a...,1
16,0007e25b2121310b,bye don t look come or think of comming ba...,1
42,001810bf8c45bf5f,you are gay or antisemmitian archangel whit...,1
43,00190820581d90ce,fuck your filthy mother in the ass dry,1


In [11]:
from sklearn.model_selection import train_test_split, KFold, cross_val_score
from sklearn.metrics import f1_score, precision_score, recall_score, precision_recall_curve, fbeta_score, confusion_matrix
from sklearn.metrics import roc_auc_score, roc_curve

from sklearn.linear_model import LogisticRegression
from sklearn.svm import LinearSVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

In [25]:
X = toxic_done.comment_text
y = toxic_done['toxic']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

tfv = TfidfVectorizer(ngram_range=(1,1), stop_words='english')

X_train_fit = tfv.fit_transform(X_train) 
X_test_fit = tfv.transform(X_test) 

randfor = RandomForestClassifier(n_estimators=100, random_state=42)

randfor.fit(X_train_fit, y_train)
randfor.predict(X_test_fit)

array([0, 1, 1, ..., 1, 1, 1])

In [ ]:
comment1 = ['you work is atrocious']
comment2 = ['What the dog doing']

In [ ]:
comment1_vect = tfv.transform(comment1)
randfor.predict_proba(comment1_vect)[:,1]

array([1.])

In [ ]:
comment2_vect = tfv.transform(comment2)
randfor.predict_proba(comment2_vect)[:,1]

array([0.16036935])

In [ ]:
X = toxic_done.comment_text
y = toxic_done['toxic']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)


tfv = TfidfVectorizer(ngram_range=(1,1), stop_words='english')

X_train_fit = tfv.fit_transform(X_train) 
X_test_fit = tfv.transform(X_test)  

logisreg = LogisticRegression()

logisreg.fit(X_train_fit, y_train)
logisreg.predict(X_test_fit)

array([0, 1, 1, ..., 1, 1, 1])

In [ ]:
comment1_vect = tfv.transform(comment1)
logisreg.predict_proba(comment1_vect)[:,1]

array([0.98335725])

In [ ]:
comment2_vect = tfv.transform(comment2)
logisreg.predict_proba(comment2_vect)[:,1]

array([0.44816824])

In [17]:
from sklearn.neighbors import KNeighborsClassifier

In [18]:
x = toxic_done.comment_text
y = toxic_done['toxic']

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.3, random_state=42)


tfv = TfidfVectorizer(ngram_range=(1,1), stop_words='english')

x_train_fit = tfv.fit_transform(x_train)  
x_test_fit = tfv.transform(x_test)  

knn = KNeighborsClassifier(n_neighbors=100)



knn.fit(x_train_fit, y_train)
knn.predict_proba(x_test_fit)

array([[0.71, 0.29],
       [0.07, 0.93],
       [0.19, 0.81],
       ...,
       [0.12, 0.88],
       [0.44, 0.56],
       [0.45, 0.55]])

In [12]:
import pickle
def pickle_model(df, label):
    
    X = df.comment_text
    y = df[label]

    tfv = TfidfVectorizer(ngram_range=(1,1), stop_words='english')
    
    X_vect = tfv.fit_transform(X)  

    with open(r"{}.pkl".format(label + '_vect'), "wb") as f:   
        pickle.dump(tfv, f)   
        
    randomforest = RandomForestClassifier(n_estimators=100, random_state=42)
    randomforest.fit(X_vect, y)

    with open(r"r_{}.pkl".format(label + '_model'), "wb") as f:  
        pickle.dump(randomforest, f)

    logisreg = LogisticRegression()
    logisreg.fit(X_vect, y)

    with open(r"l_{}.pkl".format(label + '_model'), "wb") as f:  
        pickle.dump(logisreg, f)

    svm = LinearSVC()
    svm.fit(X_vect, y)


    with open(r"s_{}.pkl".format(label + '_model'), "wb") as f:  
        pickle.dump(svm, f)

In [22]:
def Knn_model(df, label):
    
    X = df.comment_text
    y = df[label]

    tfv = TfidfVectorizer(ngram_range=(1,1), stop_words='english')
    
    X_vect = tfv.fit_transform(X)    
        
    knn = KNeighborsClassifier(n_neighbors=30)
    knn.fit(X_vect, y)

    with open(r"k_{}.pkl".format(label + '_model'), "wb") as f:  
        pickle.dump(knn, f)

    

In [23]:
datalist = [ toxic_done, severe_done, obscene_done, insult_done, threat_done, identity_hate_done]
label = ['toxic', 'severe_toxic', 'obscene', 'insult', 'threat', 'identity_hate']


for i,j in zip(datalist,label):
    Knn_model(i, j)